<H1> Click run all to see full results! <H1>

In [1]:
from Bayes_News_Categorizers import News_Models as nm
from Path_Finder_Algorithms import A_star_city_search as ascs
from Path_Finder_Algorithms import UC_city_search as uccs
from Traveling_Salesman_Algorithm import TSP_with_GA as twga

In [3]:
import pandas as pd
from Path_Finder_Algorithms import CityMaps as cm

employeeType = 0
office = 0
city = ''
category = ''
while employeeType not in (1, 2):
    try:
        employeeType = int(input("Enter 1 for Executive Correspondent or 2 for Reporter: "))
    except ValueError:
        print("Please enter a valid number.")

if employeeType == 1:            
    selected_cities = twga.select_cities_to_visit()

    # Create filtered distance matrix and cities list
    filtered_matrix, filtered_cities = twga.create_filtered_data(selected_cities)

    print(f"\nOptimizing route through: {', '.join(selected_cities)}")
    print("Starting and ending at Phoenix (Head Office)")

    # GA parameters
    population_size = 50
    generations = 100
    mutation_rate = 0.1

    # Run the genetic algorithm
    best_path, best_distance = twga.genetic_algorithm(filtered_matrix, population_size, generations, mutation_rate)

    # Display results
    if best_path:
        route_cities = ["Phoenix"] + [filtered_cities[i] for i in best_path] + ["Phoenix"]
        print(f"\nBest route found: {' -> '.join(route_cities)}")
        print(f"Total distance: {best_distance} miles")
    else:
        print("\nNo cities selected to visit. Route: Phoenix -> Phoenix")
        print("Total distance: 0 miles")

else:
    city = input("Please enter the city you work in: ")
    category = input("Please enter the news category you report on: ")
    model_name = 'Bayes_News_Categorizers/nb_combined_model.pkl'
    vectorizer_name = 'Bayes_News_Categorizers/combined_vectorizer.pkl'
    loaded_model, loaded_vectorizer = nm.load_model(model_name, vectorizer_name)
    
    # Read the Daily_News.csv file
    daily_news = pd.read_csv('Daily_News.csv')
    
    # Categorize the news using the NB model
    news_texts = daily_news['Text'].fillna('')  # Handle any NaN values
    
    # Vectorize the text data
    text_vectorized = loaded_vectorizer.transform(news_texts)
    
    # Predict categories for all news articles
    predicted_categories = loaded_model.predict(text_vectorized)
    
    # Add predicted categories to the dataframe
    daily_news['Predicted_Category'] = predicted_categories
    
    
    print(f"\nPredicted categories: {set(predicted_categories)}")
    
    # Check for matches with reporter's city and category
    print(f"\nLooking for news in '{city}' with category '{category}'...")
    
    # Filter news articles that match the reporter's location and category
    # Create an empty list to store matching news
    matching_rows = []
    
    # Go through each row in the daily_news dataframe
    for index, row in daily_news.iterrows():
        location_matches = row['Location'].lower() == city.lower()
        category_matches = row['Predicted_Category'].lower() == category.lower()
    
        if location_matches and category_matches:
            matching_rows.append(index)
    
    matching_news = daily_news.loc[matching_rows]
    
    if len(matching_news) > 0:
        print(f"\nFound {len(matching_news)} matching news article(s)")
        print("\nMatching articles:")
        for index, row in matching_news.iterrows():
            print(f"In index:{index} - {row['Summary']} - {row['Location']} - Category: {row['Predicted_Category']})")
        
        # Determine reporter's office based on city location
        reporter_office = None
        cities_graph = None
        heuristics = None

        for laCity in cm.Los_Angeles_cities.keys():
            if city.lower() == laCity.lower():
                reporter_office = "Los Angeles"
                cities_graph = cm.Los_Angeles_cities
                heuristics = cm.Los_Angeles_heuristics
        
        for sdCity in cm.San_Diego_cities.keys():
            if city.lower() == sdCity.lower():
                reporter_office = "San Diego"
                cities_graph = cm.San_Diego_cities
                heuristics = cm.San_Diego_heuristics
        
        for lvCity in cm.Las_Vegas_cities.keys():
            if city.lower() == lvCity.lower():
                reporter_office = "Las Vegas"
                cities_graph = cm.Las_Vegas_cities
                heuristics = cm.Las_Vegas_heuristics
        
        for phxCity in cm.Phoenix_cities.keys():
            if city.lower() == phxCity.lower():
                reporter_office = "Phoenix"
                cities_graph = cm.Phoenix_cities
                heuristics = cm.Phoenix_heuristics
        
        if reporter_office:
            print(f"\nReporter office region: {reporter_office}")
            
            # Find shortest path for each matching news location
            for index, row in matching_news.iterrows():
                news_location = row['Location']
                
                # Check if the news location exists in the graph
                location_found = False
                actual_location = None
                for graph_city in cities_graph.keys():
                    if graph_city.lower() == news_location.lower():
                        actual_location = graph_city
                        location_found = True
                        break
                
                if location_found:
                    print(f"\n----- Finding shortest path to {news_location} ---")
                    
                    # Use A* search algorithm (preferred for shortest path)
                    try:
                        path, cost = ascs.a_star_search(cities_graph, reporter_office, actual_location, heuristics)
                        print(f"A* Search Result:")
                        print(f"Shortest path: {' -> '.join(path)}")
                        print(f"Total cost: {cost}")
                    except Exception as e:
                        print(f"A* search failed: {e}")
                        
                        # Fallback to UCS if A* fails
                        try:
                            cost, path = uccs.uniform_cost_search(cities_graph, reporter_office, actual_location)
                            print(f"UCS Search Result (fallback):")
                            print(f"Shortest path: {' -> '.join(path)}")
                            print(f"Total cost: {cost}")
                        except Exception as e2:
                            print(f"Both A* and UCS search failed: {e2}")
                else:
                    print(f"News location '{news_location}' not found in the {reporter_office} region map.")
        else:
            print(f"City '{city}' not found in any of the available region maps.")
    else:
        print("No matching news articles found for your city and category.")
        print("\nAvailable locations in news data:")
        print(daily_news['Location'].unique())
        print(f"\nPredicted categories in news data:")
        print(sorted(set(predicted_categories)))
    



Predicted categories: {'SPORTS', 'PARENTING', 'STYLE & BEAUTY', 'WORLD NEWS', 'FOOD & DRINK', 'TRAVEL', 'POLITICS', 'BUSINESS', 'WELLNESS', 'ENTERTAINMENT'}

Looking for news in 'Mesa' with category 'Business'...

Found 1 matching news article(s)

Matching articles:
In index:1 - Mesa tech company announces new hiring spree - Mesa - Category: BUSINESS)

Reporter office region: Phoenix

----- Finding shortest path to Mesa ---
A* Search Result:
Shortest path: Phoenix -> Tempe -> Mesa
Total cost: 16
